In [1]:
# importing pacakges
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [13]:
# selecting directory
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = r'C:\Users\BERLIN\Desktop\Image-Augmentation-using-GAN-master\dataset'
width=256
height=256
depth=3

In [14]:
# converting all image into numpy array (function)
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [15]:
# loading all images from folder and subfolder
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:200]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing aflatoxin ...
[INFO] Processing anthracnose ...
[INFO] Processing Apricot monilia ...
[INFO] Processing Black Root Rot ...
[INFO] Processing Black Rot ...
[INFO] Processing Botrytis ...
[INFO] Processing carnesiyam ...
[INFO] Processing downy mildew ...
[INFO] Processing leaf rust ...
[INFO] Processing mosaic virus ...
[INFO] Processing Oedema ...
[INFO] Processing Orange Rust ...
[INFO] Processing Powdery Mildew ...
[INFO] Processing Purple Blotch ...
[INFO] Processing sooty blotch ...
[INFO] Image loading completed


In [16]:
# checking the length of the list
image_size = len(image_list)

In [18]:
# importing pickle for loading label file
import pickle
# binarizer for normallization
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [19]:
# checking all classes
print(label_binarizer.classes_)

['Apricot monilia' 'Black Root Rot' 'Black Rot' 'Botrytis' 'Oedema'
 'Orange Rust' 'Powdery Mildew' 'Purple Blotch' 'aflatoxin' 'anthracnose'
 'carnesiyam' 'downy mildew' 'leaf rust' 'mosaic virus' 'sooty blotch']


In [21]:
import numpy as np
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [22]:
# spliting our dataset into train and test split
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

[INFO] Spliting data to train, test


In [23]:
# generating data generator for validation
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [24]:
# our squeezenet model
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [25]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

[INFO] training network...


In [26]:
# training our model
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

## training steps was carry on for 25 iterations..i u need means just increase by changing epochs

c:\users\berlin\.conda\envs\opencv-env\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
6/6 [==============================] - 108s 13s/step - loss: 0.9958 - accuracy: 0.1208 - val_loss: 0.6304 - val_accuracy: 0.1176
Epoch 2/25
6/6 [==============================] - 66s 11s/step - loss: 0.8482 - accuracy: 0.1433 - val_loss: 0.6386 - val_accuracy: 0.0588
Epoch 3/25
6/6 [==============================] - 79s 14s/step - loss: 0.8422 - accuracy: 0.2075 - val_loss: 0.7995 - val_accuracy: 0.0588
Epoch 4/25
6/6 [==============================] - 81s 13s/step - loss: 0.8175 - accuracy: 0.1761 - val_loss: 0.6803 - val_accuracy: 0.0588
Epoch 5/25
6/6 [==============================] - 82s 14s/step - loss: 0.7349 - accuracy: 0.1859 - val_loss: 0.7844 - val_accuracy: 0.0588
Epoch 6/25
6/6 [==============================] - 73s 12s/step - loss: 0.7529 - accuracy: 0.2419 - val_loss: 0.5900 - val_accuracy: 0.0784
Epoch 7/25
6/6 [==============================] - 71s 12s/step - loss: 0.6964 - accuracy: 0.2976 - val_loss: 0.4567 - val_accuracy: 0.0588
Epoch 8/25
6/6 [==========

In [29]:
# evaluating our model
scores = model.evaluate(x_test, y_test)


2/2 [==============================] - 3s 1s/step - loss: 0.3137 - accuracy: 0.0196


In [ ]:
# save the model to disk
print("[INFO] Saving model...")
pickle.dump(model,open('cnn_model.pkl', 'wb'))

In [30]:
%matplotlib inline

In [31]:
def mention_output(x):
    plt.imshow(array_to_img(x))
    return label_binarizer.classes_[np.argmax(model.predict(np.expand_dims(x, axis = 0)))]

In [14]:
from keras.models import model_from_json
from keras.models import load_model

In [15]:
import cv2

In [6]:
img3 = cv2.imread(r'C:\Users\BERLIN\Desktop\Image-Augmentation-using-GAN-master\dataset\train\aflatoxin\4z.jpg')

In [ ]:
# saving our model
model.save("model_squeeze.h5")

In [13]:
import numpy as np
import keras 
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
# loading our model
model = load_model('model_squeeze.h5')

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 85, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 85, 85, 64)        0

In [18]:
from keras import models
from keras import layers

from keras import optimizers
from keras.preprocessing import image
from IPython.display import Image, display
# from learntools.deep_learning.decode_predictions import decode_predictions
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement learntools
ERROR: No matching distribution found for learntools


In [19]:
# compiling our model (categorical_crossentropy for multiclass, binary_crossentropy for 2 classes)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# Testing with input image

In [22]:
# this is ur imput image need to classify 
img = r'C:\Users\BERLIN\Desktop\Image-Augmentation-using-GAN-master\dataset\train\leaf rust\download (1).jpg' # imput is as image
img = np.full((100,80,3), 12, dtype = np.uint8)
img = cv2.resize(img,(256,256))
img = np.reshape(img,[1,256,256,3])
#plt.imshow(img)
#plt.show()
classes = model.predict_classes(img)

print (classes)


[9]


In [21]:
dict = {0:'Apricot monilia',1:'carnesiyam',2:'aflatoxin',3:'alternaria',4:'anthracnose',5:'Black Root Rot',6:'Black Rot',7:'Botrytis',8:'downy mildew',9:'leaf rust',10:'mosaic virus',11:'Oedema',12:'Orange Rust',13:'Powdery Mildew',14:'Purple Blotch',15:'sooty blotch'}
dict[classes.item()] # output as our class

'leaf rust'

In [ ]:
# correctly predicted
#'leafrust 'isthe output of our model classification